In [ ]:
%load_ext autoreload
%autoreload 2
import cvxpy
from matplotlib import pyplot as plt
import numpy as np
from src.components.synthetic_house import SyntheticHouse
from src.components.synthetic_microgrid import SyntheticMicrogrid
from src.environments.simple_microgrid import SimpleMicrogrid
from src.rl.a2c.d_simple_microgrid import Agent
from src.components.battery import Battery, BatteryParameters
from src.utils.tools import set_all_seeds, load_config, plot_metrics

# Microgrid

## Functions

In [ ]:
def solver(house: SyntheticHouse, n: int = 24):

    battery = cvxpy.Variable(n+1)
    action = cvxpy.Variable(n)
    consumption = cvxpy.Variable(n)

    constraints = []

    # Battery
        # Starts in 0.1
    constraints.append(battery[0] == house.battery.soc_min)
        # Max and min batteries
    for i in range(n+1):
        constraints.append(battery[i] <= house.battery.soc_max)
        constraints.append(battery[i] >= house.battery.soc_min)


    # Action / Batteryn't

    for i in range(n):
        constraints.append(action[i] <= 1)
        constraints.append(action[i] >= -1)


    # Transition
    obj = 0

    for i in range(n):
        
        constraints.append(action[i] <= house.battery.p_charge_max)
        constraints.append(action[i] <= house.battery.p_discharge_max)
        # Update battery
        constraints.append(battery[i+1] == battery[i] + action[i] * house.battery.efficiency)
        # Update net 
        constraints.append(consumption[i] == house.demand[i]-house.pv_gen[i] + action[i] * house.battery.efficiency)


        obj += cvxpy.maximum(consumption[i] * (house.price[i] + house.emission[i]),0) 
        obj += cvxpy.maximum(-consumption[i] * house.price[i] * house.grid_sell_rate,0)  


    objective = cvxpy.Minimize(obj)
    prob = cvxpy.Problem(objective, constraints)
    res = prob.solve()

    return res, battery.value, action.value

In [ ]:
def get_all_actions(env: SimpleMicrogrid, mode : str = 'train') -> np.ndarray:
    # Set mode, train, eval, test
    env.mg.change_mode(mode)
    state, reward, _, _ = env.reset()

    # Create arrays to hold score, battery SOC and Action for all houses
    rewards, battery_values, action_values = [],[],[]
    
    # Same for all houses
    for house in env.mg.houses:
        reward, batt, action = solver(house)
        
        rewards.append(reward)
        battery_values.append(batt)
        action_values.append(action)
    rewards = np.array(rewards)
    battery_values = np.array(battery_values)
    action_values = np.array(action_values)

    return rewards, battery_values, action_values

In [ ]:
def loop_env(env: SimpleMicrogrid, action_values: np.ndarray, mode : str = 'train') -> np.ndarray:
    env.mg.change_mode(mode)

    state, reward, done, _ = env.reset()
    rewards = []
    time_step = 0

    # Cycle the entire episode with already computed actions by solver
    while not done:
        time_step = env.mg.current_step
        state,reward,done,_ = env.step(action_values[:,time_step])
        time_step += 1
        # Save rewards, mean over all batches
        rewards.append(reward.mean(axis=1))
    # Get the sum over steps and mean from 6 houses
    rewards = np.array(rewards).sum(axis=0).mean()
    # Calculate Score with actions
    return rewards, env.mg.get_houses_metrics() # output price, emissions ]

## Run this

In [ ]:
set_all_seeds(0)
# create environment,m save array of houses
config = load_config("d_a2c_mgE1")
config = config['train']
env = SimpleMicrogrid(config=config['env'])

In [ ]:
set_all_seeds(0)
# Train
mode = 'train'
rewards_t, battery_values, action_values = get_all_actions(env, mode)
rewards_t_env, train_metrics = loop_env(env, action_values, mode)
print("Mean rewards",rewards_t.mean(), rewards_t_env)
print('train ', train_metrics)

# Eval
mode = 'eval'
rewards_e, battery_values, action_values = get_all_actions(env, mode)
print(rewards_e.mean())
eval_metrics = loop_env(env, action_values, mode)
print('eval= ', eval_metrics)

# Test
mode = 'test'
rewards_s, battery_values, action_values = get_all_actions(env, mode)
print(rewards_s.mean())
test_metrics = loop_env(env, action_values, mode)
print('test= ', test_metrics)

In [ ]:
np.array(train_metrics).mean(axis=1), np.array(eval_metrics).mean(axis=1), np.array(test_metrics).mean(axis=1)

In [ ]:
set_all_seeds(0)

model = "d_a2c_mgE1"
config = load_config(model)
config = config['train']
my_env = SimpleMicrogrid(config=config['env'])

agent = Agent(env=my_env, config = config)
results_ag = agent.train()
results_ag['test'] = agent.test()
agent.wdb_logger.finish()

In [ ]:
plot_metrics(metrics=results_ag)

# House

### Set env vars

In [ ]:
set_all_seeds(0)

# Same env obs as test simple env

config = load_config("c_a2c")
config = config['train']

mg = SyntheticHouse(config=config['env'])
demand = mg.demand
pv = mg.pv_gen
price_s = mg.price
emission = mg.emission
price_b = price_s / 4

In [ ]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
    
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

### Adv battery calc (not working)

In [ ]:
set_all_seeds(0)
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't
# for i in range(n):
#     constraints.append(action[i] <= 0.9)
#     constraints.append(action[i] >= -0.9)

# Transition
obj = 0

for i in range(n):
    # Max and min battery charge 
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery SOC
    # self.soc = self.soc + (p_charge * self.efficiency - p_discharge / self.efficiency) / self.capacity

    constraints.append(battery[i+1] == battery[i] + ((action[i] * battery_params["efficiency"])/battery_params["efficiency"])/battery_params["capacity"] )
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value, consumption.value, price_s

### Real dataset

In [ ]:
n = 24

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery
    constraints.append(battery[i+1] == battery[i] + action[i] * battery_params["efficiency"])
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

### Test Dataset

In [ ]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4

In [ ]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value